In [19]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")

   Resolving package versions...
  No Changes to `C:\Users\almen\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\almen\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\almen\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\almen\.julia\environments\v1.10\Manifest.toml`


In [20]:
using CSV
using DataFrames
using Statistics

### Part 2: Double/Debiased machine learning in observational data

In this part, we will be using observational data for computing the average treatment effect of the same program as in Part 1. This data is constructed by taking the treatment group from the same dataset as in Part 1, but constructing the control group from a different dataset; that is, the entirety of the control is comprised of observations from the Current Population Survey. Therefor, we may not have comparable treatment and control groups. To tackle this issue, we can use Double/Debiased machine learning.


2.1. Load the data (1 points).

In [33]:
data_part2 = CSV.read("C:/Users/almen/OneDrive/Escritorio/subir repor5/CausalAI-Course/Labs/Assignment/Assignment_5/data/observational/biased_control.csv", DataFrame)

Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78,agesq,agecube,educsq,u74,u75,interaction1,re74sq,re75sq
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,0,45,11,0,0,1,1,21.5167,25.2436,25.5647,2025,91125,121,0,0,236.683,462.967,637.237
2,0,21,14,0,0,0,0,3.17597,5.85256,13.4961,441,9261,196,0,0,44.4636,10.0868,34.2525
3,0,38,12,0,0,1,0,23.039,25.1308,25.5647,1444,54872,144,0,0,276.468,530.796,631.555
4,0,48,6,0,0,1,1,24.9944,25.2436,25.5647,2304,110592,36,0,0,149.966,624.718,637.237
5,0,18,8,0,0,1,1,1.6693,10.7276,9.86087,324,5832,64,0,0,13.3544,2.78655,115.082
6,0,22,11,0,0,1,1,16.3658,18.4493,25.5647,484,10648,121,0,0,180.023,267.838,340.376
7,0,48,10,0,0,1,1,16.8046,16.3546,18.0593,2304,110592,100,0,0,168.046,282.396,267.473
8,0,18,11,0,0,0,1,1.14421,3.62003,15.7393,324,5832,121,0,0,12.5863,1.30922,13.1046
9,0,48,9,0,0,1,1,25.8623,25.2436,25.5647,2304,110592,81,0,0,232.761,668.86,637.237


In [34]:
numeric_columns = [:age, :educ, :re74, :re75, :re78, :agesq, :agecube, :educsq, :interaction1, :re74sq, :re75sq]
x_numeric = select(data_part2, numeric_columns)

for col in numeric_columns
    data_part2[!, col] = (data_part2[!, col] .- mean(data_part2[!, col])) ./ std(data_part2[!, col])
end

first(data_part2, 5)  # Muestra las primeras 5 filas del DataFrame


Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78,agesq,agecube,educsq,u74,u75,interaction1,re74sq,re75sq
,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Float64
1,0,1.07457,-0.351563,0,0,1,1,0.794352,1.25965,1.11831,1.02738,0.935689,-0.46859,0,0,0.518815,0.682789,1.48892
2,0,-1.10003,0.694461,0,0,0,0,-1.11353,-0.82245,-0.129604,-0.994422,-0.882194,0.649806,0,0,-0.964524,-1.05545,-0.951227
3,0,0.440311,-0.0028882,0,0,1,0,0.952714,1.24753,1.11831,0.285796,0.13065,-0.125615,0,0,0.825831,0.943132,1.46592
4,0,1.34639,-2.09494,0,0,1,1,1.15612,1.25965,1.11831,1.38349,1.36798,-1.73611,0,0,-0.150372,1.30362,1.48892
5,0,-1.37186,-1.39759,0,0,1,1,-1.27026,-0.298995,-0.505492,-1.14376,-0.958339,-1.31857,0,0,-1.20459,-1.08347,-0.624129


2.2. Group comparisons (1.5 points). For the treatment and control group separately, report summary statistics of three variables of your choice. Can you spot any big differences between the treatment and control groups?

In [35]:
variables = [:black, :marr, :nodegree]

# Grupo control (treat = 0)
group_control = filter(:treat => ==(0), data_part2)
# Grupo tratado (treat = 1)
group_treated = filter(:treat => ==(1), data_part2)


Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78,agesq,agecube,educsq,u74,u75,interaction1,re74sq,re75sq
,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Float64
1,1,0.349702,-0.351563,1,0,1,1,-1.44391,-1.45087,-0.498339,0.190067,0.0369623,-0.46859,1,1,-1.30765,-1.09417,-1.08984
2,1,-1.00942,-1.04891,0,1,0,1,-1.44391,-1.45087,-1.1533,-0.939537,-0.851394,-1.06507,1,1,-1.30765,-1.09417,-1.08984
3,1,-0.284556,-0.0028882,1,0,0,0,-1.44391,-1.45087,1.05056,-0.40856,-0.48828,-0.125615,1,1,-1.30765,-1.09417,-1.08984
4,1,-0.556381,-0.351563,1,0,0,1,-1.44391,-1.45087,-0.748975,-0.626822,-0.650762,-0.46859,1,1,-1.30765,-1.09417,-1.08984
5,1,-0.0127312,-1.39759,1,0,0,1,-1.44391,-1.45087,-1.49516,-0.167322,-0.289823,-1.31857,1,1,-1.30765,-1.09417,-1.08984
6,1,-1.00942,-1.04891,1,0,0,1,-1.44391,-1.45087,-1.10568,-0.939537,-0.851394,-1.06507,1,1,-1.30765,-1.09417,-1.08984
7,1,-0.918815,-0.0028882,1,0,0,0,-1.44391,-1.45087,-1.52513,-0.8821,-0.817663,-0.125615,1,1,-1.30765,-1.09417,-1.08984
8,1,-0.10334,-0.351563,1,0,0,1,-1.44391,-1.45087,-0.649088,-0.250287,-0.360195,-0.46859,1,1,-1.30765,-1.09417,-1.08984
9,1,-1.00942,1.39181,1,0,0,0,-1.44391,-1.45087,-1.30136,-0.939537,-0.851394,1.54452,1,1,-1.30765,-1.09417,-1.08984


In [36]:
println("\nGrupo control (treat=0):")
for var in variables
    mean_value = mean(group_control[!, var])
    sum_value = sum(group_control[!, var])
    count_value = size(group_control, 1)
    println("Variable: $var, Mean: $mean_value, Sum: $sum_value, Count: $count_value")
end

println("\nGrupo tratado (treat=1):")
for var in variables
    mean_value = mean(group_treated[!, var])
    sum_value = sum(group_treated[!, var])
    count_value = size(group_treated, 1)
    println("Variable: $var, Mean: $mean_value, Sum: $sum_value, Count: $count_value")
end



Grupo control (treat=0):
Variable: black, Mean: 0.0735367683841921, Sum: 1176, Count: 15992
Variable: marr, Mean: 0.7117308654327164, Sum: 11382, Count: 15992
Variable: nodegree, Mean: 0.2958354177088544, Sum: 4731, Count: 15992

Grupo tratado (treat=1):
Variable: black, Mean: 0.8432432432432433, Sum: 156, Count: 185
Variable: marr, Mean: 0.1891891891891892, Sum: 35, Count: 185
Variable: nodegree, Mean: 0.7081081081081081, Sum: 131, Count: 185


In [37]:
println("\nCambios en características por grupo:")

for var in variables
    mean_control = mean(group_control[!, var])
    mean_treated = mean(group_treated[!, var])
    println("People with '$var':")
    println("  Control group: $(mean_control * 100)%")
    println("  Treated group: $(mean_treated * 100)%")
    println("  Difference: $((mean_treated - mean_control) * 100)%")
end



Cambios en características por grupo:
People with 'black':
  Control group: 7.35367683841921%
  Treated group: 84.32432432432432%
  Difference: 76.97064748590512%
People with 'marr':
  Control group: 71.17308654327164%
  Treated group: 18.91891891891892%
  Difference: -52.25416762435271%
People with 'nodegree':
  Control group: 29.58354177088544%
  Treated group: 70.8108108108108%
  Difference: 41.22726903992537%


2.3. Compute the SMD (1.5 points). Find the simple difference of means, which we can use as a naive estimate of the ATE. How does the result in this case compare to the result in point 1.2.?

In [26]:
# Media de re78 para el grupo tratado
mean_treated = mean(filter(:treat => ==(1), data_part2)[:re78])

# Media de re78 para el grupo control
mean_control = mean(filter(:treat => ==(0), data_part2)[:re78])

# ATE (Average Treatment Effect)
ate = mean_treated - mean_control

println("Mean re78 for the treated group: $(round(mean_treated, digits=2))")
println("Mean re78 for the control group: $(round(mean_control, digits=2))")
println("Average Treatment Effect (ATE): $(round(ate, digits=2))")


UndefVarError: UndefVarError: `data_part2` not defined

In [27]:
y = data_part2[!,10]
d = data_part2[!,1]
x = data_part2[:, Not([1, 10])]

UndefVarError: UndefVarError: `data_part2` not defined

In [28]:
# Generar fórmula para el modelo OLS
all_columns = join(names(data_part2)[3:end], " + ")
my_formula = "re78 ~ " * all_columns


UndefVarError: UndefVarError: `data_part2` not defined

2.4. Using DML (6 points). Use the DML procedure as we saw in the Lab, in order to find a better estimate of the ATE. You may use the doubleML packages for Python and R, but this package does not exist for Julia, so you will have to build your own procedure like we saw in class. You will be rewarded extra points for using more than one method for predictions. At the end, report the treatment effect you found, as well as the MSE for 
D
 and 
Y
 achieved by the method(s) you used.